**Model training in three ways:**

- using images obtained with `Spectrogram` in `03_preprocessing_spectrogram.ipynb`
- using spectrograms transformed by `Spectrogram` on the fly
- using spectrograms transformed by `Spectrogram` on the fly with augmentations

In [1]:
import os
import yaml
import joblib
#
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
#
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
#
import torchvision
from torchvision.io import read_image
from torchvision.transforms import ConvertImageDtype
#
import matplotlib.pyplot as plt
#
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
print(torch.__version__)
print(torchaudio.__version__)
print(torchvision.__version__)

2.3.0+cu121
2.3.0+cu121
0.18.0+cu121


In [3]:
raw_path = 'data/raw/'
data_path = 'data/spectrogram/'

In [4]:
LEARNING_RATE = 0.001

In [5]:
labels = yaml.safe_load(open('labels.yaml'))[2022]

In [6]:
X, y = zip(*labels.items())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)

In [7]:
def training(model, optimizer, criterion, dataloader):
    losses, y_true, y_score = [], [], []
    #
    model.train()
    for X, Y in dataloader:
        X, Y = X.to(device), Y.to(device, dtype=torch.float32)
        #
        output = model(X)
        loss = criterion(output.view(1), Y)
        #
        losses.append(loss.item())
        y_true.append(Y.item())
        y_score.append(torch.sigmoid(output).item())
        #
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return torch.tensor(losses).mean(), roc_auc_score(y_true, y_score)


def testing(model, optimizer, criterion, dataloader):
    losses, y_true, y_score = [], [], []
    #
    model.eval()
    with torch.inference_mode():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device, dtype=torch.float32)
            #
            output = model(X)
            loss = criterion(output.view(1), Y)
            #
            losses.append(loss.item())
            y_true.append(Y.item())
            y_score.append(torch.sigmoid(output).item())
    #
    return torch.tensor(losses).mean(), roc_auc_score(y_true, y_score)

# Modelling using images

In [8]:
device = 'cpu'

In [9]:
from utils.miscellaneous import update_stats
from utils.plot import get_figure, update_figure

In [10]:
class Podcast(Dataset):

    def __init__(self, X, y):
        self.x = X
        self.y = y
        self.cid = ConvertImageDtype(torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        image = read_image(f'{data_path}/{episode}.png')
        return self.cid(image), int(label)

In [11]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [12]:
test = Podcast(X_test, y_test)
test_dataloader = DataLoader(test)

In [13]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 16))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(4096, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #
        x = self.flatten(x)
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        #
        print('=' if self.training else '-', end='')
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [14]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [15]:
filename = '04_spectrogram_images'

In [ ]:
# !rm artifacts/04_spectrogram_images.joblib
# !rm artifacts/04_spectrogram_images.pt

In [16]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

!Model weights are loaded.
!Training stats is loaded.


In [17]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'e78b1fee-d0b5-4d2f-bcea-9ea16762ed88',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 16,
                    17, 18, 19, 20, 21, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
              'xaxis': 'x',
              'y': [0.6473167538642883, 0.6175066232681274, 0.6200255751609802,
                    0.6232159733772278, 0.6205547451972961, 0.6200083494186401,
                    0.6181317567825317, 0.607863187789917, 0.5915827751159668,
                    0.6265431046485901, 0.61553555727005, 0.6012495756149292,
                    0.5751613974571228, 0.6118099093437195, 0.7005868554115295,
                    0.6693742871284485, 0.6453839540481567, 0.6

In [ ]:
for epoch in range(se, 200):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats, filename)
    #
    joblib.dump(stats, f'artifacts/{filename}.joblib')
    if epoch % 5 == 0:
        if len(stats['testing_loss']) >= 10:
            x0 = torch.tensor(stats['testing_loss'][-10:-5]).mean()
            if x0 > torch.tensor(stats['testing_loss'][-5:]).mean():
                torch.save(
                    model.state_dict(),
                    f'artifacts/{filename}.pt'
                )
                print('!Model weights were saved.')

# Modelling `as is`

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
from utils.spectrograms import get_spectrogram
from utils.miscellaneous import update_stats
from utils.plot import get_figure, update_figure

In [18]:
class Podcast(Dataset):

    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        #
        waveform, sample_rate = torchaudio.load(f'{raw_path}/{episode}.mp3')
        #
        return get_spectrogram(waveform.mean(dim=0), sample_rate), label

In [19]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [20]:
test = Podcast(X_test, y_test)
test_dataloader = DataLoader(test)

In [25]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 16))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(1024, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self, x):
        # print('.', end='')
        shapes = [x.shape]
        # print()
        x = self.layer1(x)
        shapes.append(x.shape)
        x = self.layer2(x)
        shapes.append(x.shape)
        x = self.layer3(x)
        #
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        print(shapes, self.training)
        # print('=' if self.training else '-', end='')
        #
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [26]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [27]:
train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)

[torch.Size([1, 22051, 380]), torch.Size([1, 3674, 126]), torch.Size([1, 305, 20])] True


/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[torch.Size([1, 22051, 6836]), torch.Size([1, 3674, 2278]), torch.Size([1, 305, 378])] True
[torch.Size([1, 22051, 9316]), torch.Size([1, 3674, 3104]), torch.Size([1, 305, 516])] True
[torch.Size([1, 22051, 376]), torch.Size([1, 3674, 124]), torch.Size([1, 305, 19])] True
[torch.Size([1, 22051, 811]), torch.Size([1, 3674, 269]), torch.Size([1, 305, 44])] True
[torch.Size([1, 22051, 10663]), torch.Size([1, 3674, 3553]), torch.Size([1, 305, 591])] True
[torch.Size([1, 22051, 10717]), torch.Size([1, 3674, 3571]), torch.Size([1, 305, 594])] True
[torch.Size([1, 22051, 462]), torch.Size([1, 3674, 153]), torch.Size([1, 305, 24])] True
[torch.Size([1, 22051, 576]), torch.Size([1, 3674, 191]), torch.Size([1, 305, 31])] True
[torch.Size([1, 22051, 8935]), torch.Size([1, 3674, 2977]), torch.Size([1, 305, 495])] True
[torch.Size([1, 24001, 571]), torch.Size([1, 3999, 189]), torch.Size([1, 332, 30])] True
[torch.Size([1, 22051, 473]), torch.Size([1, 3674, 157]), torch.Size([1, 305, 25])] True
[tor

In [13]:
filename = '04_spectrogram'

In [14]:
# !rm artifacts/04_spectrogram.joblib
# !rm artifacts/04_spectrogram.pt

In [15]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

In [17]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'd75c8fd7-84f7-4255-93c1-d86de94902d8',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'red', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Testing',
              'type': 'scatter',
              'uid': 'a67c2412-66fe-4a2c-9ea1-250839a17e70',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', '

In [24]:
for epoch in range(se, 200):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats)
    #
    joblib.dump(stats, f'artifacts/{filename}.joblib')
    if epoch % 5 == 0:
        if len(stats['testing_loss']) >= 10:
            x0 = torch.tensor(stats['testing_loss'][-10:-5]).mean()
            if x0 > torch.tensor(stats['testing_loss'][-5:]).mean():
                torch.save(
                    model.state_dict(),
                    f'artifacts/{filename}.pt'
                )
                print('!Model weights were saved.')

=

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)



=============================================================================--------------------------
Epoch: 0 | Training loss: 0.6839 | Testing loss: 0.6688 | Training AUC: 0.5463 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 1 | Training loss: 0.6566 | Testing loss: 0.6394 | Training AUC: 0.4830 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 2 | Training loss: 0.6341 | Testing loss: 0.6189 | Training AUC: 0.5779 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 3 | Training loss: 0.6154 | Testing loss: 0.5984 | Training AUC: 0.3945 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 4 | Training loss: 0.6063 | Testing loss: 0.5902 | Training AUC: 

KeyboardInterrupt: 

## + augmentations

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [31]:
from utils.augmentations import augment, masking
from utils.spectrograms import get_spectrogram
from utils.miscellaneous import update_stats
from utils.plot import get_figure, update_figure

In [32]:
class Podcast(Dataset):

    def __init__(self, X, y, clipping=True, training=True):
        self.x = X
        self.y = y
        self.clipping = clipping
        self.training = training

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        #
        waveform, sample_rate = torchaudio.load(f'{raw_path}/{episode}.mp3')
        waveform = waveform.mean(dim=0)
        #
        if self.training:
            waveform = augment(waveform, sample_rate, self.clipping)
        spectrogram = get_spectrogram(waveform, sample_rate)
        if self.training:
            spectrogram = masking(spectrogram)
        #
        return spectrogram, label

In [33]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [34]:
test = Podcast(X_test, y_test, training=False)
test_dataloader = DataLoader(test)

In [38]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 8))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(256, 64)
        self.linear2 = nn.Linear(64, 1)

    def forward(self, x):
        # 
        shapes = [x.shape]
        # print()
        x = self.layer1(x)
        shapes.append(x.shape)
        x = self.layer2(x)
        shapes.append(x.shape)
        x = self.layer3(x)
        #
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        print(shapes, self.training)
        # print('=' if self.training else '-', end='')
        #
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [39]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [40]:
train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)

[torch.Size([1, 22051, 227]), torch.Size([1, 3674, 75]), torch.Size([1, 305, 11])] True
[torch.Size([1, 22051, 227]), torch.Size([1, 3674, 75]), torch.Size([1, 305, 11])] True
[torch.Size([1, 22051, 227]), torch.Size([1, 3674, 75]), torch.Size([1, 305, 11])] True
[torch.Size([1, 22051, 227]), torch.Size([1, 3674, 75]), torch.Size([1, 305, 11])] True
[torch.Size([1, 24001, 209]), torch.Size([1, 3999, 69]), torch.Size([1, 332, 10])] True
[torch.Size([1, 22051, 227]), torch.Size([1, 3674, 75]), torch.Size([1, 305, 11])] True


KeyboardInterrupt: 

In [19]:
filename = '04_spectrogram_augmented'

In [ ]:
# !rm artifacts/04_spectrogram_augmented.joblib
# !rm artifacts/04_spectrogram_augmented.pt

In [20]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

In [21]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'edc7e996-62e5-4959-af8c-b5de8b11f1bd',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'red', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Testing',
              'type': 'scatter',
              'uid': 'd1c68424-6ef3-4598-bb1a-22cce02b78dd',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', '

In [24]:
for epoch in range(se, 200):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats)
    #
    joblib.dump(stats, f'artifacts/{filename}.joblib')
    if epoch % 5 == 0:
        if len(stats['testing_loss']) >= 10:
            x0 = torch.tensor(stats['testing_loss'][-10:-5]).mean()
            if x0 > torch.tensor(stats['testing_loss'][-5:]).mean():
                torch.save(
                    model.state_dict(),
                    f'artifacts/{filename}.pt'
                )
                print('!Model weights were saved.')

=

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)



=============================================================================--------------------------
Epoch: 0 | Training loss: 0.6839 | Testing loss: 0.6688 | Training AUC: 0.5463 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 1 | Training loss: 0.6566 | Testing loss: 0.6394 | Training AUC: 0.4830 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 2 | Training loss: 0.6341 | Testing loss: 0.6189 | Training AUC: 0.5779 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 3 | Training loss: 0.6154 | Testing loss: 0.5984 | Training AUC: 0.3945 | Testing AUC: 0.5000
==============================================================================--------------------------
Epoch: 4 | Training loss: 0.6063 | Testing loss: 0.5902 | Training AUC: 

KeyboardInterrupt: 